# Eat Safe, Love

## Notebook Set Up

In [1]:
from pymongo import MongoClient
import pandas as pd
from pprint import pprint

In [2]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [3]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [4]:
# review the collections in our database
print(db.list_collection_names())

['establishments']


In [5]:
# assign the collection to a variable
establishments = db['establishments']

## Part 3: Exploratory Analysis
Unless otherwise stated, for each question: 
* Use `count_documents` to display the number of documents contained in the result.
* Display the first document in the results using `pprint`.
* Convert the result to a Pandas DataFrame, print the number of rows in the DataFrame, and display the first 10 rows.

### 1. Which establishments have a hygiene score equal to 20?

In [6]:
# Find the establishments with a hygiene score of 20
query = {"scores.Hygiene": 20}

# Use count_documents to display the number of documents in the result
hygiene_20_count = establishments.count_documents(query)
print("Number of establishments with a hygiene score of 20:", hygiene_20_count)

# Display the first document in the results using pprint
first_document = establishments.find_one(query)
pprint(first_document)

Number of establishments with a hygiene score of 20: 0
None


In [7]:
# Convert the result to a Pandas DataFrame
result_df = pd.DataFrame(list(establishments.find(query)))

# Display the number of rows in the DataFrame
num_rows = len(result_df)
print("Number of rows in the DataFrame:", num_rows)

# Display the first 10 rows of the DataFrame
print("First 10 rows of the DataFrame:")
print(result_df.head(10))

Number of rows in the DataFrame: 0
First 10 rows of the DataFrame:
Empty DataFrame
Columns: []
Index: []


### 2. Which establishments in London have a `RatingValue` greater than or equal to 4?

In [8]:
# Find the establishments with London as the Local Authority and has a RatingValue greater than or equal to 4.
query = {"LocalAuthorityName": {"$regex": "London", "$options": "i"}, "RatingValue": {"$gte": 4}}

# Use count_documents to display the number of documents in the result
london_rating_count = establishments.count_documents(query)
print("Number of establishments in London with a RatingValue greater than or equal to 4:", london_rating_count)

# Display the first document in the results using pprint
first_document = establishments.find_one(query)
pprint(first_document)

Number of establishments in London with a RatingValue greater than or equal to 4: 0
None


In [9]:
# Convert the result to a Pandas DataFrame
result_df = pd.DataFrame(list(establishments.find(query)))

# Display the number of rows in the DataFrame
num_rows = len(result_df)
print("Number of rows in the DataFrame:", num_rows)

# Display the first 10 rows of the DataFrame
print("First 10 rows of the DataFrame:")
print(result_df.head(10))

Number of rows in the DataFrame: 0
First 10 rows of the DataFrame:
Empty DataFrame
Columns: []
Index: []


### 3. What are the top 5 establishments with a `RatingValue` rating value of 5, sorted by lowest hygiene score, nearest to the new restaurant added, "Penang Flavours"?

In [11]:
# Search within 0.01 degree on either side of the latitude and longitude.
# Rating value must equal 5
# Sort by hygiene score

degree_search = 0.01
latitude = 51.490142
longitude = 0.08384

query = {
    "geocode.latitude": {"$gte": latitude - degree_search, "$lte": latitude + degree_search},
    "geocode.longitude": {"$gte": longitude - degree_search, "$lte": longitude + degree_search},
    "RatingValue": 5
}

sort = [("scores.Hygiene", 1)]
limit = 5

results = establishments.find(query).sort(sort).limit(limit)

# Print the results
for result in results:
    pprint(result)

In [12]:
# Convert result to Pandas DataFrame
result_list = list(results)
result_df = pd.DataFrame(result_list)

# Display the DataFrame
print(result_df)

Empty DataFrame
Columns: []
Index: []


### 4. How many establishments in each Local Authority area have a hygiene score of 0?

In [13]:
# Create a pipeline that:
# 1. Matches establishments with a hygiene score of 0
# 2. Groups the matches by Local Authority
# 3. Sorts the matches from highest to lowest

pipeline = [
    {"$match": {"scores.Hygiene": 0}},
    {"$group": {"_id": "$LocalAuthorityName", "count": {"$sum": 1}}},
    {"$sort": {"count": -1}}
]

result = establishments.aggregate(pipeline)

# Print the number of documents in the result
result_count = sum(1 for _ in result)
print("Number of documents in the result:", result_count)

# Print the first 10 results
result = establishments.aggregate(pipeline)
for i, doc in enumerate(result):
    if i < 10:
        pprint(doc)

Number of documents in the result: 0


In [14]:
# Convert the result to a Pandas DataFrame
result = list(establishments.aggregate(pipeline))
result_df = pd.DataFrame(result)

# Display the number of rows in the DataFrame
num_rows = len(result_df)
print("Number of rows in the DataFrame:", num_rows)

# Display the first 10 rows of the DataFrame
print("First 10 rows of the DataFrame:")
print(result_df.head(10))


Number of rows in the DataFrame: 0
First 10 rows of the DataFrame:
Empty DataFrame
Columns: []
Index: []
